# Final Notebook

# 1. Setting Up the Dependencies

- Navigate to Project Directory.
- Create a virtual environment using `virtualenv`. Activate the virtual environment afterwards.
```bash
virtualenv .tire-scan-venv
source .tire-scan-venv/bin/activate
```
- Install the required dependencies listed in the `requirements.txt` file using `pip`:
```bash
pip install -r requirements.txt
```

- Navigate to the notebook and use the virtual environment.

In [1]:
# imports
import os, requests

# constants
from constants import googlesearch_api_key, googlesearch_cx, googlesearch_api_url, gemini_api_key
# set google api variables
os.environ["GOOGLE_CSE_ID"] = googlesearch_cx
os.environ["GOOGLE_API_KEY"] = googlesearch_api_key

# utils
from utils import remove_tags_to_ignore, extract_tags_to_use, translate_text
# import the htmt tags to parse (or ignore)
from constants import tags_to_use, tags_to_ignore
# other constants
from constants import max_chunk_size, chunk_intersection_size
# import prompts
from prompts import custom_qa_prompt, custom_retrival_prompt, output_schema

from bs4 import BeautifulSoup
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

/Users/sampadk04/Desktop/Coding/Active_Projects/Michelin_Project/.tire-scan-venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# URL Extraction

- `extract_top_urls`:

The function returns a JSON object containing a list of relevant queries along with additional information. Subsequently, the `extract_url_list` function is commonly employed to extract the URLs from this JSON object, providing easy access to the web resources associated with the provided EAN code. Also the actual number of results retrieved is capped at the minimum of top_k and 10 to ensure a manageable and relevant set of queries.

### Workflow:

1. The function initializes a `GoogleSearchAPIWrapper` to interact with the Google Search API efficiently.

2. It defines an inner function `top_results(query)` to retrieve the top results for a given query. This function is then utilized by the `GoogleSearchAPIWrapper` as a custom tool named "Google Search Snippets."

3. The main function forms a query string in the format "EAN <ean_code>" and uses the custom tool to fetch results from the Google Search API.

4. The results are returned as a JSON object containing information about the top results, including titles, links, and snippets.

In [2]:
def extract_top_urls(ean_code, top_k = 5):
    search = GoogleSearchAPIWrapper()

    def top_results(query):
        return search.results(query, min(top_k, 10))
    
    tool = Tool(
        name="Google Search Snippets",
        description="Search Google for recent results.",
        func=top_results,
    )

    query = f"EAN {ean_code}"
    results = tool.run(query)
    return results

- `extract_url_list`:

This helper function takes the results JSON object from `extract_top_urls` and extracts a list of URLs. It iterates over the result items and collects the `'link'` attribute into a list, providing a convenient way to access the URLs for further processing.

In [3]:
def extract_url_list(results):
    """
    Extracts the URLs and corresponding snippets from a list of dictionaries.

    Args:
        results (list): A list of dictionaries containing 'link' and 'snippet' keys.

    Returns:
        list: A list of tuples, where each tuple contains the URL and snippet extracted from the input dictionaries.
    """
    url_list = []
    for url_dict in results:
        # extract links and corresponding snippets
        url, snippet = url_dict['link'], url_dict['snippet']
        url_list.append((url, snippet))
        print(f"URL: {url} \nSnippet: {snippet} \n\n")
    
    return url_list

- `scrape`:

The `scrape` function is responsible for extracting HTML content from a list of URLs using the `AsyncChromiumLoader`. It opens a headless Chromium instance to load the URLs and then extracts the HTML content along with metadata. The HTML content undergoes a series of preprocessing steps, including removing unnecessary tags using `remove_tags_to_ignore`, extracting relevant tags with `extract_tags_to_use`, and optional translation using Azure Translator.

In [4]:
# from langchain_community.document_loaders import AsyncChromiumLoader, AsyncHtmlLoader
# import nest_asyncio
# nest_asyncio.apply()

# session = requests.Session()
# session.trust_env = False
# # have to circumvent michelin proxies temporariliy to scrape
# # https://stackoverflow.com/questions/36906985/switch-off-proxy-in-requests-library

# def scrape(urls, tags_to_use=tags_to_use, tags_to_ignore=tags_to_ignore, translation=False):
#     loader = AsyncChromiumLoader(urls)
#     #loader = AsyncHtmlLoader(urls)
#     docs = loader.load()

#     for doc in docs:
#         html_content = doc.page_content

#         print(html_content[:100])

#         # preprocess the html_content
#         soup = BeautifulSoup(html_content, 'html.parser')

#         # remove tags to ignore
#         soup = remove_tags_to_ignore(soup, tags_to_ignore)

#         # extract tags to use
#         text = extract_tags_to_use(soup, tags_to_use)

#         if translation:
#             text_limit = 50000
#             # translate text using azure translator
#             text = translate_text(text[:text_limit])
        
#         # update the doc
#         doc.page_content = text
#     return docs

In [5]:
def scrape2(url_list, tags_to_use=tags_to_use, tags_to_ignore=tags_to_ignore, translation=False):
    docs = []
    text_limit = 50000

    session = requests.Session()
    session.trust_env = False
    # have to circumvent michelin proxies temporariliy to scrape
    # https://stackoverflow.com/questions/36906985/switch-off-proxy-in-requests-library

    for (url, snippet) in url_list:
        # use requests
        try:
            response = session.get(url)
            html_content = response.text

            # preprocess the html_content
            soup = BeautifulSoup(html_content, 'html.parser')

            # remove tags to ignore
            # soup = remove_tags_to_ignore(soup, tags_to_ignore)

            # extract tags to use
            text = extract_tags_to_use(soup, tags_to_use)

            if translation:
                # translate text using azure translator
                text = translate_text(text[:text_limit])

            # store this in a Document object
            doc = Document(page_content=text + f" Snippet: {snippet}", metadata={'source': url})
            docs.append(doc)
        except:
            print(f"Error scraping {url}")

    return docs

In [6]:
def scrape3(url_list, tags_to_use=tags_to_use, tags_to_ignore=tags_to_ignore, translation=False):
    docs = []
    text_limit = 50000

    session = requests.Session()
    session.trust_env = False
    # have to circumvent michelin proxies temporariliy to scrape
    # https://stackoverflow.com/questions/36906985/switch-off-proxy-in-requests-library

    for (url, snippet) in url_list:
        # use requests
        try:
            response = session.get(url)
            html_content = response.text

            # preprocess the html_content
            soup = BeautifulSoup(html_content, 'html.parser')

            # remove tags to ignore
            # soup = remove_tags_to_ignore(soup, tags_to_ignore)

            # extract tags to use
            text = extract_tags_to_use(soup, tags_to_use)

            if translation:
                # translate text using azure translator
                text = translate_text(text[:text_limit])

            # store this in a Document object
            #cdoc = Document(page_content=text + f" Snippet: {snippet}", metadata={'source': url})
            docs.append(f"URL: {url}\nSNIPPET: {snippet}" + text)
        except:
            print(f"Error scraping {url}")

    return docs

- `split_docs`

The `split_docs` function divides the overall document into smaller chunks while retaining metadata. It uses `tiktoken_encoder` to split the content into chunks of a specified token size limit, with optional intersection between consecutive chunks.

In [7]:
def split_docs(docs, max_chunk_size=max_chunk_size, chunk_intersection_size=chunk_intersection_size):
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=max_chunk_size, chunk_overlap=chunk_intersection_size
    )

    splits = splitter.split_documents(docs)

    return splits

# Check Validity of EAN

- `is_ean_valid`:

The `is_ean_valid` function plays a crucial role in determining the validity of an EAN (European Article Number) code by assessing the presence of relevant Google search results associated with it. This function encapsulates the logic to identify whether an EAN code is valid or invalid based on the outcome of the `extract_top_urls` function.

This function serves as a valuable tool in the EAN validation process, allowing users to quickly identify and flag invalid EAN codes based on the availability of relevant Google search results.

### Workflow:

1. The function attempts to retrieve relevant URLs associated with the provided EAN code using the `extract_top_urls` function.

2. If the `extract_top_urls` function encounters an exception during execution (such as network issues or API limitations), the function catches the exception and assigns a predefined result indicating no good Google search result was found.

3. The function then compares the obtained results with the predefined invalid result. If the results match, indicating no valid URLs were found, the function concludes that the EAN code is invalid.

4. The function returns a boolean value: `True` if the EAN code is valid (at least one relevant URL found) and `False` if it is invalid (no relevant URLs found).


In [8]:
def is_ean_valid(ean_code):
    invalid_ean_result = [{'Result': 'No good Google Search Result was found'}]
    try:
        results = extract_top_urls(ean_code)
    except Exception as e:
        print(e.message, e.args)
        results = invalid_ean_result
    
    if results == invalid_ean_result:
        return False
    else:
        return True

In [9]:
ean_code = 8903094003235
# false eg : 4250463418034
# true eg: 8903094003235, 3528700139716
is_ean_valid(8903094003235)

True

In [10]:
# extract docs
scraped_docs = scrape2(extract_url_list(extract_top_urls(ean_code, top_k=5)))

URL: https://rotyre.com/EAN/8903094003235/BKT/da 
Snippet: Producent, BKT. EAN, 8903094003235. Producentens artikelnummer, 15722535. Handelsbetegnelse eller varebetegnelse, TR 135 8PR (TT). 


URL: https://www.pokryshkin.pro/en/12-4-32-bkt-tr-135-8pr-tt 
Snippet: agriculture tyre 12.4-32 BKT TR-135 8PR TT delivery within Ukraine | 12.4-32 BKT TR-135 8PR TT tractor tyre | buy, reviews, price, delivery. 


URL: https://shop.bohnenkamp-baltic.com/riepas/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=ee&___from_store=lv 
Snippet: ... EAN: 8903094003235. Rehv 12.4-32 8PR BKT TR 135 124A6 R1 TT. Lugupeetud kliendid! Kui Te näete endale vajaliku toote laoseisu veerus "0", siis palun võtke ... 


URL: https://www.heuver.com/product/b12400032bk0813500/12-4-32-bkt-tr-135-124a6-121a8-8pr-tt 
Snippet: EAN code. 8903094003235. Brand. BKT. Profile. TR-135. L/S. 124A6/121A8. Ply rating. 8. TL/TT. TT. Application. Diagonal. Radial/diagonal. Diagonal. Recommended ... 


URL: https://shop.bohnenkamp-b

In [11]:
scraped_docs_text = scrape3(extract_url_list(extract_top_urls(ean_code, top_k=5)))

URL: https://rotyre.com/EAN/8903094003235/BKT/da 
Snippet: Producent, BKT. EAN, 8903094003235. Producentens artikelnummer, 15722535. Handelsbetegnelse eller varebetegnelse, TR 135 8PR (TT). 


URL: https://www.pokryshkin.pro/en/12-4-32-bkt-tr-135-8pr-tt 
Snippet: agriculture tyre 12.4-32 BKT TR-135 8PR TT delivery within Ukraine | 12.4-32 BKT TR-135 8PR TT tractor tyre | buy, reviews, price, delivery. 


URL: https://shop.bohnenkamp-baltic.com/riepas/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=ee&___from_store=lv 
Snippet: ... EAN: 8903094003235. Rehv 12.4-32 8PR BKT TR 135 124A6 R1 TT. Lugupeetud kliendid! Kui Te näete endale vajaliku toote laoseisu veerus "0", siis palun võtke ... 


URL: https://www.heuver.com/product/b12400032bk0813500/12-4-32-bkt-tr-135-124a6-121a8-8pr-tt 
Snippet: EAN code. 8903094003235. Brand. BKT. Profile. TR-135. L/S. 124A6/121A8. Ply rating. 8. TL/TT. TT. Application. Diagonal. Radial/diagonal. Diagonal. Recommended ... 


URL: https://shop.bohnenkamp-b

# Checking EAN Context

- `check_ean_context`:

The `check_ean_context` function is a pivotal step in the process of determining whether an EAN (European Article Number) code is associated with tyres. This involves scraping and splitting HTML content from extracted URLs, storing them in a vector store, and querying Language Models (LLMs) to ascertain the association.

This function serves as a valuable tool for contextual analysis, allowing users to determine the nature of the association of an EAN code with tyres through intelligent querying of Language Models.

### Workflow:

1. The function splits the HTML content into smaller chunks using the `split_docs` function while retaining metadata.

2. It initializes an Azure Chat-based OpenAI Language Model (LLM) for querying and obtaining responses.

3. The Azure OpenAI Embeddings model is configured to create embeddings for the text chunks using the Azure Text Embedding deployment.

4. A vector store is created using the Chroma library, incorporating the embeddings and document splits.

5. The function constructs a retrieval chain to query the LLM for the association of the EAN code with tyres. The result is obtained by processing the retrieved context.

6. The vector store is cleared after processing to maintain efficiency.

In [12]:
def check_ean_context(ean_code, docs, max_chunk_size=2500, chunk_intersection_size=200, k=5):
    splits = split_docs(docs, max_chunk_size=max_chunk_size, chunk_intersection_size=chunk_intersection_size)

    llm = ChatGoogleGenerativeAI(
        model="gemini-pro",
        temperature=0.0,
        max_output_tokens=20,
        google_api_key=gemini_api_key
    )

    # define the embedding model
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        task_type="retrieval_document",
        google_api_key=gemini_api_key
    )

    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

    # retrieve and generate using the relevant snippets
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    # helper function to join splits
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # define the rag chain
    ean_context_rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | custom_qa_prompt
        | llm
        | StrOutputParser()
    )

    query = f"Is this EAN code {ean_code} associated with a tyre? Return Yes or No and nothing else."

    is_ean_context_valid = ean_context_rag_chain.invoke(query)

    vectorstore.delete_collection()

    if is_ean_context_valid == "Yes":
        return True
    else:
        return False

In [13]:
check_ean_context(ean_code, scraped_docs)

True

# Algorithm 1: RAG Pipeline

- `rag_llm_pipeline`

The `rag_llm_pipeline` function serves as a critical component for extracting specific information such as diameter, width, brand, model, and vehicle type associated with a validated EAN (European Article Number) code. This extraction process involves querying a Language Model (LLM) separately for each attribute, with the model retrieving relevant information based on matched context collected from a vector store.

### Workflow:

1. The function splits the HTML content into smaller chunks using the `split_docs` function while retaining metadata.

2. It initializes an Azure Chat-based OpenAI Language Model (LLM) for querying and obtaining responses.

3. The Azure OpenAI Embeddings model is configured to create embeddings for the text chunks using the Azure Text Embedding deployment.

4. A vector store is created using the Chroma library, incorporating the embeddings and document splits.

5. The function constructs a retrieval chain to query the LLM for each specified attribute separately. The LLM responses are collected in an output dictionary.

6. The vector store is cleared after processing to maintain efficiency.

In [14]:
def rag_llm_pipeline(ean_code, docs, llm_model, max_chunk_size, chunk_intersection_size, k, output_schema):
    splits = split_docs(docs, max_chunk_size=max_chunk_size, chunk_intersection_size=chunk_intersection_size)

    llm = ChatGoogleGenerativeAI(
        model="gemini-pro",
        temperature=0.0,
        max_output_tokens=20,
        google_api_key=gemini_api_key
    )

    # define the embedding model
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        task_type="retrieval_document",
        google_api_key=gemini_api_key
    )

    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

    # retrieve and generate using the relevant snippets
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    # helper function to join splits
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # define the rag chain
    rag_qa_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | custom_retrival_prompt
        | llm
        | StrOutputParser()
    )
    
    # define the output_dict
    llm_output = {}

    for key in output_schema:
        try:
            query = output_schema[key]
            output = rag_qa_chain.invoke(query)
            llm_output[key] = output
        except Exception as e:
            print("Exception:", e)
            llm_output[key] = "NA"

    vectorstore.delete_collection()

    return llm_output

In [15]:
llm_model = "gpt-35-turbo"
max_chunk_size = 1024
chunk_intersection_size = 128
k = 5

rag_llm_pipeline(
    ean_code,
    scraped_docs,
    llm_model=llm_model,
    max_chunk_size=max_chunk_size,
    chunk_intersection_size=chunk_intersection_size,
    k=k,
    output_schema=output_schema
)

{'Diameter': 'NA',
 'Width': 'NA',
 'Brand': 'BKT',
 'Model': 'TR 135',
 'Vehicle Type': 'NA'}

# Algorithm 2: Prompt Updating

In [16]:
def prompt_llm_pipeline(
        ean_code,
        docs_text,
        context_size=6000,
        context_intersection=200,
        ):
    # set the prompts
    system_prompt = "AI is an intelligent retriever designed to help users extract information on tyres based on the raw_text scraped from the web. Instructions: \n - The extraction should include information like - diameter - width - height - price - brand - model - vehicle type of tyre.\n - Strictly follow the output format for filling in the data. \n - The summary should have bullet points for each of the above mentioned label. \n Do not include information not present in the scraped data. Skip the label not available."

    user_prompt_init_ = "Here is the output format based on the system instruction. AI should this format strictly for output: \n\nOutput Format: \n\nAccording to the scraped data, the following information is available:\n\n- Diameter: [insert diameter here]\n- Width: [insert width here]\n- Height: [insert height here]\n- Price: [insert price or price range here]\n- Brand: [insert brand here]\n- Model: [insert model here]\n- Vehicle Type of Tyre: [insert vehicle type here]\n\n"

    assistant_prompt_repeat_ = "Here is the output retrieved from the previous chunk of scraped data. Update this output following these instructions:\n -Here are the outputs from previous iteration, so make sure to retain all the non-redundant information.\n - Change value of any key having \"Not available\", only after getting evidence from current chunk.\n - Follow this output format strictly for output.\n - Update value to multiple values or range depending on updated information from the chunk.\n\n So, here is the output from previous iteration:\n\n"

    # define the llm model
    llm = ChatGoogleGenerativeAI(
        model="gemini-pro",
        temperature=0.0,
        max_output_tokens=100,
        google_api_key=gemini_api_key
    )   
    
    # join the docs_text as a single string
    context = "\n\n".join(docs_text)

    # print("CONTEXT: ", context)
    # print("LENGTH", len(context))

    # initialize an empty list to store the chunks
    chunks = []

    # split the scraped content into chunks of size `context_size`
    for i in range(0, len(context), (context_size - context_intersection)):
        chunks.append(context[i:i + context_size])
    
    # handle each chunk separately

    # for the initial prompt use system_prompt + chunk_prompt_init + assistant_prompt_init
    # for the remaining loop prompts use system_prompt + chunk_prompt_repeat + assistant_prompt_repeat
        
    # initialize a string to hold the llm output from previous iteration
    llm_output = ""

    for i, chunk in enumerate(chunks):
        if i == 0:
            chunk_prompt_init = "\n\n Web Scraped Chunk No. " + str(i+1) + ":\n\n"
            chunk_prompt_init += "\n\n" + chunk + "\n\n"

            # manually create the prompts (message type prompting still not supported in gemini-pro)
            system_message_0 = f"This is a system message that provides the general instructions for you (AI) and the nature of the conversation : {system_prompt}. "
            user_message_0 = f"This is a user message that provides the chunk of scraped data as context : {chunk_prompt_init}. " + f"\n{user_prompt_init_}"

            llm_query = system_message_0 + user_message_0

            # messages = ChatPromptTemplate.from_messages([
            #     ("system", system_prompt),
            #     ("user", chunk_prompt_init),
            #     ("assistant", assistant_prompt_init_)
            # ])
        else:
            chunk_prompt_repeat = "\n\n Web Scraped Chunk No. " + str(i+1) + ":\n\n"
            chunk_prompt_repeat += "\n\n" + chunk + "\n\n"

            assistant_prompt_repeat_ += "\n\n" + llm_output + "\n\n"

            system_message_ = f"This is a system message that provides the general instructions for you (AI) and the nature of the conversation : {system_prompt}. "
            user_message_ = f"This is a user message that provides the chunk of scraped data as context : {chunk_prompt_repeat}. "
            assistant_message_ = f"This is an AI message that provides initial output for AI to update based on context : {assistant_prompt_repeat_}."
            user_message_0 = f"\n{user_prompt_init_}"

            llm_query = system_message_ + user_message_ + assistant_message_ + user_message_0

            # messages = ChatPromptTemplate.from_messages([
            #     ("system", system_prompt),
            #     ("user", chunk_prompt_repeat),
            #     ("assistant", assistant_prompt_repeat_),
            #     ("assistant", assistant_prompt_init_)
            # ])
        
        # run the llm model using the messages prompt and store the output
        llm_output = llm.invoke(llm_query).content
        
        # run the llm model
        # runnable = messages | llm | StrOutputParser()
        # llm_output = runnable.invoke()

    return llm_output

In [17]:
docs_text = scraped_docs_text
context_size = 6000
context_intersection = 200

llm_output = prompt_llm_pipeline(
    ean_code,
    docs_text,
    context_size=context_size,
    context_intersection=context_intersection
)

print(llm_output)

According to the scraped data, the following information is available:

- Diameter: 32
- Width: 12.4
- Height: Not Available
- Price: Not Available
- Brand: BKT
- Model: TR 135 8PR (TT)
- Vehicle Type of Tyre: Not Available


In [23]:
# testing with google translated context for scraped documents for EAN: 8903094003235

context = """
URL: https://rotyre.com/EAN/8903094003235/BKT/en\n
  +352 / 27 69 39 0 org@rotyre.com Facebook LinkedIn B2B Shop Contact FAQ Registering Team General Terms and Conditions Impressum Datenschutz Manufacturer BKT EAN 8903094003235 Manufacturer's article number 15722535 Trade name or product name TR 135 8PR (TT) Tire dimension designation 12.4/0 R32 Tire type Summer Tire for demanding snow conditions No EU guidelines 1222/2009K WHEELS TEAM CONTACT FAQ REGISTRATION LOGIN Rotyre S.à r.l. website and the effectiveness of our advertising campaigns. You can either choose which cookies you want to accept or accept all cookies. More about this Necessary Functional Statistics Marketing Accept all cookies\n
\n
URL: https://shop.bohnenkamp-baltic.com/riepas/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=ee&___from_store=lv\n
SNIPPET: ... EAN: 8903094003235. Rehv 12.4-32 8PR BKT TR 135 124A6 R1 TT. Lugupeetud kliendid! If you see "0", then you can see the "0" item, so please see "0" in the list. Need ei ole siduvad, vaid only informative. Tarnija ei võa nende dättega sässät säätä säätät. Mistahes otseste või pävävete sääääsä, khajutasuäääää või käsäkahju eest mistahes öääägikel alustel on väätega sääää piires välistatud as a result of Vastutus siinse teabea sääää. «Bohnenkamp» OÜ Lõõtsa 12, Ülemiste City, Tallinn, 11415, Eesti Tel.: +372 65 000 21 e-mail: info@bohnenkamp.ee www.bohnenkamp.ee «Bohnenkamp» OÜ Lõõtsa 12, Ülemiste City, Tallinn, 11415 , Eesti Tel.: +372 65 000 21 e-mail: info@bohnenkamp.ee www.bohnenkamp.ee * Nõutud väljad Kas te gruvise abi?Klikkige umël olevale özilemärgile, et avada e-poe sämäti juhised. 12.4-32 8PR BKT TR 135 124A6 R1 TT VAATA LISAKS Tootenumber / Täistekst Suurus / Otsingukood Info Minu konto Või meiega ogyotäs Palun valige kliendiepakkimien, kulime te te välität selle toote lisada: Ostukorv Soovide sämät (0) 0 Sisene EN LV LT RU Rehvid Sisekummid Veljed Rattad Masinaehitus Tarvikud Eripakkumised Minu pood Teenus Home >> Rehvid >> Lisa ja ava pakkumine Lisa ja ava pakkumine Lisa Lisa Katkesta Katkesta Ilma garantiita foto EAN: 8903094003235 rim rim rim rim rim bulls rim rim tube rim rim rim rim rim rim bulls rim rim rim rim tyre «Bohnenkamp» OÜ Lõõtsa 12, Ülemiste City, Tallinn, 11415, Eesti Tel.: +372 65 000 21 e-mail: info@bohnenkamp.ee www.bohnenkamp.ee @ Saada Saada Bohnenkamp - Profil für Profis Ostukorv Soovide List (0) Sisene EN LV LT RU Rehvid Sisekummid Veljed Rattad Masinaehitus Tarvikud Eripakkumised Minu pood Teenus siia Attreve Rehwhite rim 10.00X32 (-) 8/221/275 ET0 SILVER RAL9006 STARCO ONE PART 011900 - rim 10.00X32 (-) 10/281 /335 ET-12 YELLOW RAL1018 STARCO 021477 - rim 10.00X32 6/163/222.3 ET-104 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 10/281/335 ET0 ORANGE RAL2002 STARCO 020711 - rim 10.00X32 (-) 8/221 /275 Ø20 ET-60 YELLOW RAL1018 TITAN - bulls 12.4-32 8PR BKT TR 171 124A6 R1W+ TT rim 10.00X32 (-) 8/221/275 ET50 ORANGE RAL2002 STARCO 010688 - rim 10X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS tube 11.2-32 Nexen TR218A 11.2/10-32, 12.4/11-32, 320/70-32, 360/70-32 rim 10.00X32 (-) 8/221/275 ET115 RED RAL3020 STARCO 020318 - rim 10.00X32 Ø17 ADJUSTABLE YELLOW RAL1018 TEJWHEELS COMBY rim 10.00X32 10/93/140 A2/21,5 ET30.5 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 8/221/275 ET90 YELLOW RAL1018 STARCO 010466 - rim 10.00X32 (- ) 10/281/335 ET140 YELLOW RAL1018 STARCO 020956 - rim 10.00X32 (-) 10/281/335 Ø21 ET190 YELLOW RAL1018 STARCO 628486 - bulls 320/85R32 (12.4R32) BKT AGRIMAX RT 855 126A8 /126B R1 W TL 12.4R32 rim 10.00X32 (-) 8/221/275 Ø20 ET-20 RED RAL3020 STARCO 012064 - rim 10.00X32 ADJUSTABLE SILVER RAL9006 TEJWHEELS COMBY rim 10.00X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS TEJ01019 rim 10.00X32 00X32 (- ) 10/281/335 ET50 ORANGE RAL2002 STARCO 020877 - rim 10.00X32 12/390/430 Ø17 ET-177 SILVER RAL9006 TEJWHEELS COMBY TEJ00527-525 bulls 12.4-32 8PR BKT FARM SPECIAL 124A6 R1 TT Tehnilised andmed Downloads Rekvisiidid Üldtimiduses Andmekaitse Os tukorv Kliendipakumene 15722535 Rehvid Sisekummid Veljed Rattad Masinaehitus Tarvikud Eripakkumised Minu pood Teenus Atvning >> Rehvid >> 12.4-32 8PR BKT TR 135 124A6 R1 TT rim 10.00X32 (-) 8/221/275 ET0 SILVER RAL9006 STARCO ONE PART 011900 - rim 10.00X32 ( -) 10/281/335 ET-12 Yellow RAL1018 Starco 021477-RIM 10.00x32 6/1222.3 ET-104 Silver RAL9006 TEJWHEELS RIM 10.00X32 (-) 10/281/335 ET0 Orange RAL2002 Starco 020711-RIM 10.00x32 ( -) 8/221/275 Ø20 ET-60 YELLOW RAL1018 TITAN - bulls 12.4-32 8PR BKT TR 171 124A6 R1W+ TT rim 10.00X32 (-) 8/221/275 ET50 ORANGE RAL2002 STARCO 010688 - rim 10X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS tube 11.2-32 Nexen TR218A 11.2/10-32, 12.4/11-32, 320/70-32, 360/70-32 rim 10.00X32 (-) 8/221/275 ET115 RED RAL3020 STARCO 020318 - rim 10.00X32 Ø17 ADJUSTABLE YELLOW RAL1018 TEJWH EELS COMBY rim 10.00X32 10/93/140 A2/21,5 ET30.5 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 8/221/275 ET90 YELLOW RAL1018 STARCO 010466 - rim 10.00X32 (-) 10/281/335 ET140 YELLOW RAL1018 STARCO 020956 - rim 10.00X32 (-) 10/281/335 Ø21 ET190 YELLOW RAL1018 STARCO 628486 - tyre 320/85R32 (12.4R32) BKT AGRIMAX RT 855 126A8 /126B R1W TL 12.4R32 rim 10.00X32 (-) 8/221/275 Ø20 ET-20 RED RAL3020 STARCO 012064 - rim 10.00X32 ADJUSTABLE SILVER RAL9006 TEJWHEELS COMBY rim 10.00X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS TEJ01019 rim 10.00X32 (-) 10/281/335 ET50 ORANGE RAL2002 STARCO 020877 - rim 10.00X32 12/390/430 Ø17 ET-177 SILVER RAL9006 TEJWHEELS COMBY TEJ00527-525 tyre 12.4-32 8PR BKT FARM SPECIAL 124A6 R1 TT Tehnilised andmed Downloads Tootekood15722535Rehvimõõt12.4-32TL / TTTTLI / SI124A6PR8MudelTR 135TootjaBKTSoovitatav velgW 11lubatud velgW 9 ; W 10Laius, mm325Läbimõõt, mm1360SLR, мм639RC, mm3998Kandevõime (kg) / km / h 11600 / 30Rõhk bar2.3Kaal (kg)48.74000Mustri sügavus (mm)34TRA codeR1Maht, m30.45000Kõik andmed põhinevad tootja tehnilistel andmetel. Need ei ole siduvad, vaid üksnes informatiivsed.Tarnija ei võta nende andmetega seoses mingit vastutust. Vastutus siinse teabe kasutamise tagajärjel tekkinud mistahes otseste või kaudsete kahjustuste, kahjutasunõuete või kõrvalkahju eest mistahes õiguslikel alustel on seadusega lubatud piires välistatud. Rekvisiidid Üldtingimused Andmekaitse Ostukorv Kliendipakkumine *Nimi Telefon *E-posti aadress *Märkus Tootekood 15722535 Rehvimõõt 12.4-32 TL / TT TT LI / SI 124A6 PR 8 Mudel TR 135 Tootja BKT Soovitatav velg W 11 lubatud velg W 9 ; W 10 Laius, mm 325 Läbimõõt, mm 1360 SLR, мм 639 RC, mm 3998 Kandevõime (kg) / km / h 1 1600 / 30 Rõhk bar 2.3 Kaal (kg) 48.74000 Mustri sügavus (mm) 34 TRA code R1 Maht, m3 0.45000\n
\n
URL: https://www.heuver.com/product/b12400032bk0813500/12-4-32-bkt-tr-135-124a6-121a8-8pr-tt\n
SNIPPET: EAN code. 8903094003235. Brand. BKT. Profile. TR-135. L/S. 124A6/121A8. Ply rating. 8. TL/TT. TT. Application. Diagonal. Radial/diagonal. Diagonal. Recommended ...Home Home Agricultural 12.4-32 BKT TR-135 124A6/121A8 8PR TT Home Home Agricultural 12.4-32 BKT TR-135 124A6/121A8 8PR TT This BKT 12.4-32 diagonal agricultural tyre will meet your needs! You have thus selected a pattern depth of 34 mm and a weight of 48.74 kg. It should preferably be used with a pressure of 2.3 bar. Zwedenweg 17 7772 TC Hardenberg The Netherlands Both we and third parties use cookies on our website.We use cookies to track statistics, save your preferences, for marketing purposes, and to personalise advertisements. Read more in our privacy statement 12.4-32 BKT TR-135 124A6/121A8 8PR TT Contact data We use cookies Back Necessary Preferences Analysis Advertising Products Truck Agriculture OTR Knowledge It's all round News About Heuver Contact Products Truck Agriculture OTR Knowledge It's all round News About Heuver Contact Home Home Agricultural Home Home Agricultural Login Register Why Heuver Our history Terms & Conditions Transport Conditions Knowledge News More About Heuver Login Register Forgot password Bonusshop It's all round Guarantees Claim form More My Heuver +31 (0)523 850 850 info@heuver.com More Contact data © 2024 Heuver.com Privacy statement Cookie statement Cookie settings Disclaimer Read more in our privacy statement Products Back Truck Agriculture OTR Truck Agriculture OTR Knowledge It's all round News About Heuver Contact Truck Agriculture OTR Knowledge It's all round News About Heuver Contact Why Heuver Our history Terms & Conditions Transport Conditions Knowledge News Login Register Forgot password Bonusshop It's all round Guarantees Claim form +31 (0)523 850 850 info@heuver.com Zwedenweg 17 7772 TC Hardenberg The Netherlands\n
\n
URL: https://shop.bohnenkamp-baltic.com/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=lt&___from_store=default\n
SNIPPET: 15722535 EAN: 8903094003235. Pneumatinė padanga 12.4-32 8PR BKT TR 135 124A6 R1 TT. Gerbiami klientai! Jeigu Jums reikalingų prekių atsargų stulpelyje matote ...pavyzdžiui 9524 padangos 9.5 24 dydžiui diagonalinė ar radialinė Krepšelis Pageidavimų sąrašas (0) Log In Šiuose puslapiuose visa informacija pagrįsta techninėmis gamintojo charakteristikomis. Turinys gali būti naudojamas tik informaciniais tikslais. Tiekėjas už šią informaciją neprisiima jokios atsakomybės . Atsakomybė už tiesioginius ar netiesioginius nuostolius, ieškiniai dėl žalos ar netiesioginės įvairaus pobūdžio žalos, susijusios su šios informacijos panaudojimu, negalimi. UAB «Bohnenkamp»Ukmergės g. 364 -15, Vilnius, Lietuva, LT-14188 Tel.: +370 52332611, +370 52332355 e-mail: info@bohnenkamp.lt www.bohnenkamp.lt UAB «Bohnenkamp»Ukmergės g. 364 -15, Vilnius, Lietuva, LT-14188 Tel.: +370 52332611, +370 52332355 e-mail: info@bohnenkamp.lt www.bohnenkamp.lt * Privalomas laukelis Jums reikalinga pagalba?Norėdami peržiūrėti vaizdinę medžiagą, paaiškinančią kaip naudotis internetine parduotuve, spustelėkite piktogramą, esančią dešinėje. 12.4-32 8PR BKT TR 135 124A6 R1 TT Related products Product number / Full text Size / Search code Information My account Contact Please select the customer offer to which you want to add this product Cart Wish list (0) 0 Log In EN EE LV RU Tires Cameras Discs Wheels Machine industry Other Special offers My shop Services First >> Add and open an offer Add and open an offer Add Add Cancel Cancel Photos EAN: 8903094003235 rim rim rim rim rim tire rim rim tube rim rim rim rim rim rim tire rim rim rim rim rim tire UAB «Bohnenkamp» Ukmergės st. 364 -15, Vilnius, Lithuania, LT-14188 Tel.: +370 52332611, +370 52332355 e-mail: info@bohnenkamp.lt www.bohnenkamp.lt @ Send Send Bohnenkamp - Profil für Profis Cart Wish list (0) Log In EN EE LV RU Tires Chambers Discs Wheels Machine industry Other Special offers My shop Services hier First rim 10.00X32 (-) 8/221/275 ET0 SILVER RAL9006 STARCO ONE PART 011900 - rim 10.00X32 (-) 10/281/335 ET0 -12 YELLOW RAL1018 STARCO 021477 - rim 10.00X32 6/163/222.3 ET-104 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 10/281/335 ET0 ORANGE RAL2002 STARCO 020711 - rim 10.00X32 (-) 8/221/275 Ø20 ET-60 YELLOW RAL1018 TITAN - tire 12.4-32 8PR BKT TR 171 124A6 R1W+ TT rim 10.00X32 (-) 8/221/275 ET50 ORANGE RAL2002 STARCO 010688 - rim 10X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS tube 11.2-32 Nexen TR218A 11.2/10-32, 12.4/11-32, 320/70-32, 360/70-32 rim 10.00X32 (-) 8/221/275 ET115 RED RAL3020 STARCO 020318 - rim 10.00X32 Ø17 ADJUSTABLE YELLOW RAL1018 TEJWHEELS COMBY rim 10.00X32 10/93/140 A2/21,5 ET30.5 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 8/221/275 ET90 YELLOW RAL1018 STARCO 010466 - rim 10.00X32 (-) 10/ 281/335 ET140 YELLOW RAL1018 STARCO 020956 - rim 10.00X32 (-) 10/281/335 Ø21 ET190 YELLOW RAL1018 STARCO 628486 - tire 320/85R32 (12.4R32) BKT AGRIMAX RT 855 126A8 /126B R1W TL 12 .4R32 rim 10.00X32 ( -) 8/221/275 Ø20 ET-20 RED RAL3020 STARCO 012064 - rim 10.00X32 ADJUSTABLE SILVER RAL9006 TEJWHEELS COMBY rim 10.00X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS TEJ01019 rim 10.00X32 (-) 1 0/ 281/335 ET50 ORANGE RAL2002 STARCO 020877 - rim 10.00X32 12/390/430 Ø17 ET-177 SILVER RAL9006 TEJWHEELS COMBY TEJ00527-525 tire 12.4-32 8PR BKT FARM SPECIAL 124A6 R1 TT Specification Downloads Properties Terms and Conditions Privacy Policy Cart mark 15722535 Tires Tubes Rims Wheels Machine industry Other Special offers My shop Services First >> 12.4-32 8PR BKT TR 135 124A6 R1 TT rim 10.00X32 (-) 8/221/275 ET0 SILVER RAL9006 STARCO ONE PART 011900 - rim 10.00X32 (-) 10/281/335 ET-12 YELLOW RAL1018 STARCO 021477 - rim 10.00X32 6/163/222.3 ET-104 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 10/281/335 ET0 ORANGE RAL2002 STARCO 020711 - rim 10.00X32 (-) 8/221/275 Ø20 ET-60 YELLOW RAL1018 TITAN - tire 12.4-32 8PR BKT TR 171 124A6 R1W+ TT rim 10.00X32 (-) 8/221/275 ET50 ORANGE RAL2002 STARCO 010688 - rim 10X32 8/152.4/203.2 A2/ 21,5 ET30 SILVER RAL9006 TEJWHEELS tube 11.2-32 Nexen TR218A 11.2/10-32, 12.4/11-32, 320/70-32, 360/70-32 rim 10.00X32 (-) 8/221/275 ET115 RED RAL3020 STARCO 020318 - rim 10.00X32 Ø17 ADJUSTABLE YELLOW RAL1018 TEJWHEELS COMBY rim 10.00X32 10/93/140 A2/21,5 ET30.5 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 8/221/275 ET90 YELLOW RAL1018 STARCO 010466 - rim 10.00 X32 (-) 10/281/335 ET140 YELLOW RAL1018 STARCO 020956 - rim 10.00X32 (-) 10/281/335 Ø21 ET190 YELLOW RAL1018 STARCO 628486 - tire 320/85R32 (12.4R32) BKT AGRIMAX RT 855 126A8 /126B R1W TL 12.4R32 rim 10.00X32 (-) 8/221/275 Ø20 ET-20 RED RAL3020 STARCO 012064 - rim 10.00X32 ADJUSTABLE SILVER RAL9006 TEJWHEELS COMBY rim 10.00X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS TEJWHEELS 1019 rim 10.00 X32 (-) 10/281/335 ET50 ORANGE RAL2002 STARCO 020877 - rim 10.00X32 12/390/430 Ø17 ET-177 SILVER RAL9006 TEJWHEELS COMBY TEJ00527-525 tire 12.4-32 8PR BKT FARM SPECIAL 124A6 R1 TT Specification Downloads SKU1572253 5 Size 12.4 -32TL / TTTTLI / SI124A6PR8Tread TR 135 ManufacturerBKTRecommended rimW 11Permissible rimW 9 ; W 10Width, mm325Diameter, mm1360SLR, mm639RC, mm3998Lifting power(kg)/ km/h 11600 / 30Pressure, bar2.3Weight kg48.74000Dug height mm34TRA codeR1Scope, m30.45000 On these pages, all information is based on the technical characteristics of the manufacturer. The content can be used for informational purposes only. The provider does not assume any responsibility for this information. Liability for direct or indirect losses, claims for damages or indirect damages of various kinds related to the use of this information is not possible. Details Terms and conditions Data protection Cart Mark *Name Phone *Email *Comment SKU 15722535 Size 12.4-32 TL / TT TT LI / SI 124A6 PR 8 Protector TR 135 Manufacturer BKT Recommended rim W 11 Permissible rim W 9 ; W 10 Width, mm 325 Diameter, mm 1360 SLR, mm 639 RC, mm 3998 Ke lifting power (kg)/ km/h 1 1600 / 30 Pressure, bar 2.3 Weight kg 48.74000 Height of ground supports mm 34 TRA code R1 Volume, m3 0.45000\n
"""






docs_text = scraped_docs_text
context_size=6000
context_intersection=200

# set the prompts
system_prompt = "AI is an intelligent retriever designed to help users extract information on tyres based on the raw_text scraped from the web. Instructions: \n - The extraction should include information like - diameter - width - height - price - brand - model - vehicle type of tyre.\n - Strictly follow the output format for filling in the data. \n - The summary should have bullet points for each of the above mentioned label. \n Do not include information not present in the scraped data. Skip the label not available."

user_prompt_init_ = "Here is the output format based on the system instruction. AI should this format strictly for output: \n\nOutput Format: \n\nAccording to the scraped data, the following information is available:\n\n- Diameter: [insert diameter here]\n- Width: [insert width here]\n- Height: [insert height here]\n- Price: [insert price or price range here]\n- Brand: [insert brand here]\n- Model: [insert model here]\n- Vehicle Type of Tyre: [insert vehicle type here]\n\n"

assistant_prompt_repeat_ = "Here is the output retrieved from the previous chunk of scraped data. Update this output following these instructions:\n -Here are the outputs from previous iteration, so make sure to retain all the non-redundant information.\n - Change value of any key having \"Not available\", only after getting evidence from current chunk.\n - Follow this output format strictly for output.\n - Update value to multiple values or range depending on updated information from the chunk.\n\n So, here is the output from previous iteration:\n\n"

# define the llm model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.0,
    max_output_tokens=100,
    google_api_key=gemini_api_key
)   

# join the docs_text as a single string
# context = "\n\n".join(docs_text)
#context = ""

# print("CONTEXT: ", context)
# print("LENGTH", len(context))

# initialize an empty list to store the chunks
chunks = []

# split the scraped content into chunks of size `context_size`
for i in range(0, len(context), (context_size - context_intersection)):
    chunks.append(context[i:i + context_size])

# handle each chunk separately

# for the initial prompt use system_prompt + chunk_prompt_init + assistant_prompt_init
# for the remaining loop prompts use system_prompt + chunk_prompt_repeat + assistant_prompt_repeat
    
# initialize a string to hold the llm output from previous iteration
llm_output = ""

for i, chunk in enumerate(chunks):
    if i == 0:
        chunk_prompt_init = "\n\n Web Scraped Chunk No. " + str(i+1) + ":\n\n"
        chunk_prompt_init += "\n\n" + chunk + "\n\n"

        # manually create the prompts (message type prompting still not supported in gemini-pro)
        system_message_0 = f"This is a system message that provides the general instructions for you (AI) and the nature of the conversation : {system_prompt}. "
        user_message_0 = f"This is a user message that provides the chunk of scraped data as context : {chunk_prompt_init}. " + f"\n{user_prompt_init_}"

        llm_query = system_message_0 + user_message_0
    else:
        chunk_prompt_repeat = "\n\n Web Scraped Chunk No. " + str(i+1) + ":\n\n"
        chunk_prompt_repeat += "\n\n" + chunk + "\n\n"

        assistant_prompt_repeat_ += "\n\n" + llm_output + "\n\n"

        system_message_ = f"This is a system message that provides the general instructions for you (AI) and the nature of the conversation : {system_prompt}. "
        user_message_ = f"This is a user message that provides the chunk of scraped data as context : {chunk_prompt_repeat}. "
        assistant_message_ = f"This is an AI message that provides initial output for AI to update based on context : {assistant_prompt_repeat_}."
        user_message_0 = f"\n{user_prompt_init_}"

        llm_query = system_message_ + user_message_ + assistant_message_ + user_message_0
    
    # run the llm model using the messages prompt and store the output
    llm_output = llm.invoke(llm_query).content

print(llm_output)

According to the scraped data, the following information is available:

- Diameter: 32
- Width: 12.4
- Height: Not Available
- Price: Not Available
- Brand: BKT
- Model: TR 135
- Vehicle Type of Tyre: Not Available


# Algorithm 3: Using `Refine` in Langchain

In [32]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter


urls = ["https://rotyre.com/EAN/8903094003235/BKT/da", "https://shop.bohnenkamp-baltic.com/riepas/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=ee&___from_store=lv", "https://www.heuver.com/product/b12400032bk0813500/12-4-32-bkt-tr-135-124a6-121a8-8pr-tt", "https://shop.bohnenkamp-baltic.com/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=lt&___from_store=default"]

loader = WebBaseLoader(urls)
docs = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=100
)

split_docs = text_splitter.split_documents(docs)

# define the llm model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.0,
    google_api_key=gemini_api_key
)

# prompt
prompt_template = """Extract information on the tyre with EAN 8903094003235 based on the text scraped from web. The output should include the information about the following attributes of the tyre:
ATTRIBUTES: [diameter, width, height, price, brand, model, vehicle type of tyre]
Follow this format strictly for the output (default values are `NA`):
OUTPUT FORMAT:
[
- Diameter: NA
- Width: NA
- Height: NA
- Price: NA
- Brand: NA
- Model: NA
- Vehicle Type of Tyre: NA
]
------------
SCRAPED TEXT:
{text}
------------
OUTPUT:"""

refine_template = (
    "Your task is to produce a final output for the required attributes in the specified format. Follow this format strictly for output\n"
    "We have provided the existing output up to a certain point following the specified format for the required attributes: {existing_answer}\n"
    "While maintaining the output format, we have the opportunity to refine the existing output"
    "(only if necessary) based on some more context below.\n"
    "------------\n"
    "NEW SCRAPED TEXT:\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the existing output for the required attributes to produce the final output, while maintaining the format."
    "If the context isn't useful, return the existing output"
)

prompt = PromptTemplate.from_template(prompt_template)
refine_prompt = PromptTemplate.from_template(refine_template)

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)

result = chain({"input_documents": split_docs}, return_only_outputs=True)

print(result["output_text"])

- Diameter: 32
- Width: 12.4
- Height: 1360
- Price: NA
- Brand: BKT
- Model: TR-135 124A6 R1 TT
- Vehicle Type of Tyre: Agricultural


In [33]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter

urls = ["https://rotyre.com/EAN/8903094003235/BKT/da", "https://shop.bohnenkamp-baltic.com/riepas/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=ee&___from_store=lv", "https://www.heuver.com/product/b12400032bk0813500/12-4-32-bkt-tr-135-124a6-121a8-8pr-tt", "https://shop.bohnenkamp-baltic.com/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=lt&___from_store=default"]

loader = WebBaseLoader(urls)
docs = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=100
)

split_docs = text_splitter.split_documents(docs)

# define the llm model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.0,
    google_api_key=gemini_api_key
)

# prompt
prompt_template = """Extract information on tyre based on the text scraped from web. The output should include the information about the following attributes of the tyre:
ATTRIBUTES: [diameter, width, height, price, brand, model, vehicle type of tyre]
Follow this format strictly for the output (default values are `NA`):
OUTPUT FORMAT:
[
- Diameter: NA
- Width: NA
- Height: NA
- Price: NA
- Brand: NA
- Model: NA
- Vehicle Type of Tyre: NA
]
------------
SCRAPED TEXT:
{text}
------------
OUTPUT:"""

refine_template = (
    "Your task is to produce a final output for the required attributes in the specified format. Follow this format strictly for output\n"
    "We have provided the existing output up to a certain point following the specified format for the required attributes: {existing_answer}\n"
    "While maintaining the output format, we have the opportunity to refine the existing output"
    "(only if necessary) based on some more context below.\n"
    "------------\n"
    "NEW SCRAPED TEXT:\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the existing output for the required attributes to produce the final output, while maintaining the format."
    "If the context isn't useful, return the existing output"
)

prompt = PromptTemplate.from_template(prompt_template)
refine_prompt = PromptTemplate.from_template(refine_template)

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)

result = chain({"input_documents": split_docs}, return_only_outputs=True)

print(result["output_text"])

- Diameter: 32
- Width: 12.4
- Height: 1360
- Price: NA
- Brand: BKT
- Model: TR-135 124A6 R1 TT
- Vehicle Type of Tyre: Agricultural


In [31]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=100
)

split_docs = text_splitter.split_documents(scraped_docs)

# define the llm model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.0,
    google_api_key=gemini_api_key
)

# prompt
prompt_template = """Extract information on tyre based on the text scraped from web. The output should include the information about the following attributes of the tyre:
ATTRIBUTES: [diameter, width, height, price, brand, model, vehicle type of tyre]
Follow this format strictly for the output:
OUTPUT FORMAT:
[
- Diameter: NA
- Width: NA
- Height: NA
- Price: NA
- Brand: NA
- Model: NA
- Vehicle Type of Tyre: NA
]
------------
SCRAPED TEXT:
{text}
------------
OUTPUT:"""

refine_template = (
    "Your task is to produce a final output for the required attributes in the specified format. Follow this format strictly for output\n"
    "We have provided the existing output up to a certain point following the specified format for the required attributes: {existing_answer}\n"
    "While maintaining the output format, we have the opportunity to refine the existing output"
    "(only if necessary) based on some more context below.\n"
    "------------\n"
    "NEW SCRAPED TEXT:\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the existing output for the required attributes to produce the final output, while maintaining the format."
    "If the context isn't useful, return the existing output"
)

prompt = PromptTemplate.from_template(prompt_template)
refine_prompt = PromptTemplate.from_template(refine_template)

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)

result = chain({"input_documents": split_docs}, return_only_outputs=True)

print(result["output_text"])

- Diameter: 1360 mm
- Width: 325 mm
- Height: NA
- Price: NA
- Brand: BKT
- Model: TR 135 8PR (TT)
- Vehicle Type of Tyre: Agricultural


# Algorithm 4: Using `Extraction Chains` in Langchain

In [22]:
context = """
URL: https://rotyre.com/EAN/8903094003235/BKT/en\n
  +352 / 27 69 39 0 org@rotyre.com Facebook LinkedIn B2B Shop Contact FAQ Registering Team General Terms and Conditions Impressum Datenschutz Manufacturer BKT EAN 8903094003235 Manufacturer's article number 15722535 Trade name or product name TR 135 8PR (TT) Tire dimension designation 12.4/0 R32 Tire type Summer Tire for demanding snow conditions No EU guidelines 1222/2009K WHEELS TEAM CONTACT FAQ REGISTRATION LOGIN Rotyre S.à r.l. website and the effectiveness of our advertising campaigns. You can either choose which cookies you want to accept or accept all cookies. More about this Necessary Functional Statistics Marketing Accept all cookies\n
\n
URL: https://shop.bohnenkamp-baltic.com/riepas/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=ee&___from_store=lv\n
SNIPPET: ... EAN: 8903094003235. Rehv 12.4-32 8PR BKT TR 135 124A6 R1 TT. Lugupeetud kliendid! If you see "0", then you can see the "0" item, so please see "0" in the list. Need ei ole siduvad, vaid only informative. Tarnija ei võa nende dättega sässät säätä säätät. Mistahes otseste või pävävete sääääsä, khajutasuäääää või käsäkahju eest mistahes öääägikel alustel on väätega sääää piires välistatud as a result of Vastutus siinse teabea sääää. «Bohnenkamp» OÜ Lõõtsa 12, Ülemiste City, Tallinn, 11415, Eesti Tel.: +372 65 000 21 e-mail: info@bohnenkamp.ee www.bohnenkamp.ee «Bohnenkamp» OÜ Lõõtsa 12, Ülemiste City, Tallinn, 11415 , Eesti Tel.: +372 65 000 21 e-mail: info@bohnenkamp.ee www.bohnenkamp.ee * Nõutud väljad Kas te gruvise abi?Klikkige umël olevale özilemärgile, et avada e-poe sämäti juhised. 12.4-32 8PR BKT TR 135 124A6 R1 TT VAATA LISAKS Tootenumber / Täistekst Suurus / Otsingukood Info Minu konto Või meiega ogyotäs Palun valige kliendiepakkimien, kulime te te välität selle toote lisada: Ostukorv Soovide sämät (0) 0 Sisene EN LV LT RU Rehvid Sisekummid Veljed Rattad Masinaehitus Tarvikud Eripakkumised Minu pood Teenus Home >> Rehvid >> Lisa ja ava pakkumine Lisa ja ava pakkumine Lisa Lisa Katkesta Katkesta Ilma garantiita foto EAN: 8903094003235 rim rim rim rim rim bulls rim rim tube rim rim rim rim rim rim bulls rim rim rim rim tyre «Bohnenkamp» OÜ Lõõtsa 12, Ülemiste City, Tallinn, 11415, Eesti Tel.: +372 65 000 21 e-mail: info@bohnenkamp.ee www.bohnenkamp.ee @ Saada Saada Bohnenkamp - Profil für Profis Ostukorv Soovide List (0) Sisene EN LV LT RU Rehvid Sisekummid Veljed Rattad Masinaehitus Tarvikud Eripakkumised Minu pood Teenus siia Attreve Rehwhite rim 10.00X32 (-) 8/221/275 ET0 SILVER RAL9006 STARCO ONE PART 011900 - rim 10.00X32 (-) 10/281 /335 ET-12 YELLOW RAL1018 STARCO 021477 - rim 10.00X32 6/163/222.3 ET-104 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 10/281/335 ET0 ORANGE RAL2002 STARCO 020711 - rim 10.00X32 (-) 8/221 /275 Ø20 ET-60 YELLOW RAL1018 TITAN - bulls 12.4-32 8PR BKT TR 171 124A6 R1W+ TT rim 10.00X32 (-) 8/221/275 ET50 ORANGE RAL2002 STARCO 010688 - rim 10X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS tube 11.2-32 Nexen TR218A 11.2/10-32, 12.4/11-32, 320/70-32, 360/70-32 rim 10.00X32 (-) 8/221/275 ET115 RED RAL3020 STARCO 020318 - rim 10.00X32 Ø17 ADJUSTABLE YELLOW RAL1018 TEJWHEELS COMBY rim 10.00X32 10/93/140 A2/21,5 ET30.5 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 8/221/275 ET90 YELLOW RAL1018 STARCO 010466 - rim 10.00X32 (- ) 10/281/335 ET140 YELLOW RAL1018 STARCO 020956 - rim 10.00X32 (-) 10/281/335 Ø21 ET190 YELLOW RAL1018 STARCO 628486 - bulls 320/85R32 (12.4R32) BKT AGRIMAX RT 855 126A8 /126B R1 W TL 12.4R32 rim 10.00X32 (-) 8/221/275 Ø20 ET-20 RED RAL3020 STARCO 012064 - rim 10.00X32 ADJUSTABLE SILVER RAL9006 TEJWHEELS COMBY rim 10.00X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS TEJ01019 rim 10.00X32 00X32 (- ) 10/281/335 ET50 ORANGE RAL2002 STARCO 020877 - rim 10.00X32 12/390/430 Ø17 ET-177 SILVER RAL9006 TEJWHEELS COMBY TEJ00527-525 bulls 12.4-32 8PR BKT FARM SPECIAL 124A6 R1 TT Tehnilised andmed Downloads Rekvisiidid Üldtimiduses Andmekaitse Os tukorv Kliendipakumene 15722535 Rehvid Sisekummid Veljed Rattad Masinaehitus Tarvikud Eripakkumised Minu pood Teenus Atvning >> Rehvid >> 12.4-32 8PR BKT TR 135 124A6 R1 TT rim 10.00X32 (-) 8/221/275 ET0 SILVER RAL9006 STARCO ONE PART 011900 - rim 10.00X32 ( -) 10/281/335 ET-12 Yellow RAL1018 Starco 021477-RIM 10.00x32 6/1222.3 ET-104 Silver RAL9006 TEJWHEELS RIM 10.00X32 (-) 10/281/335 ET0 Orange RAL2002 Starco 020711-RIM 10.00x32 ( -) 8/221/275 Ø20 ET-60 YELLOW RAL1018 TITAN - bulls 12.4-32 8PR BKT TR 171 124A6 R1W+ TT rim 10.00X32 (-) 8/221/275 ET50 ORANGE RAL2002 STARCO 010688 - rim 10X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS tube 11.2-32 Nexen TR218A 11.2/10-32, 12.4/11-32, 320/70-32, 360/70-32 rim 10.00X32 (-) 8/221/275 ET115 RED RAL3020 STARCO 020318 - rim 10.00X32 Ø17 ADJUSTABLE YELLOW RAL1018 TEJWH EELS COMBY rim 10.00X32 10/93/140 A2/21,5 ET30.5 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 8/221/275 ET90 YELLOW RAL1018 STARCO 010466 - rim 10.00X32 (-) 10/281/335 ET140 YELLOW RAL1018 STARCO 020956 - rim 10.00X32 (-) 10/281/335 Ø21 ET190 YELLOW RAL1018 STARCO 628486 - tyre 320/85R32 (12.4R32) BKT AGRIMAX RT 855 126A8 /126B R1W TL 12.4R32 rim 10.00X32 (-) 8/221/275 Ø20 ET-20 RED RAL3020 STARCO 012064 - rim 10.00X32 ADJUSTABLE SILVER RAL9006 TEJWHEELS COMBY rim 10.00X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS TEJ01019 rim 10.00X32 (-) 10/281/335 ET50 ORANGE RAL2002 STARCO 020877 - rim 10.00X32 12/390/430 Ø17 ET-177 SILVER RAL9006 TEJWHEELS COMBY TEJ00527-525 tyre 12.4-32 8PR BKT FARM SPECIAL 124A6 R1 TT Tehnilised andmed Downloads Tootekood15722535Rehvimõõt12.4-32TL / TTTTLI / SI124A6PR8MudelTR 135TootjaBKTSoovitatav velgW 11lubatud velgW 9 ; W 10Laius, mm325Läbimõõt, mm1360SLR, мм639RC, mm3998Kandevõime (kg) / km / h 11600 / 30Rõhk bar2.3Kaal (kg)48.74000Mustri sügavus (mm)34TRA codeR1Maht, m30.45000Kõik andmed põhinevad tootja tehnilistel andmetel. Need ei ole siduvad, vaid üksnes informatiivsed.Tarnija ei võta nende andmetega seoses mingit vastutust. Vastutus siinse teabe kasutamise tagajärjel tekkinud mistahes otseste või kaudsete kahjustuste, kahjutasunõuete või kõrvalkahju eest mistahes õiguslikel alustel on seadusega lubatud piires välistatud. Rekvisiidid Üldtingimused Andmekaitse Ostukorv Kliendipakkumine *Nimi Telefon *E-posti aadress *Märkus Tootekood 15722535 Rehvimõõt 12.4-32 TL / TT TT LI / SI 124A6 PR 8 Mudel TR 135 Tootja BKT Soovitatav velg W 11 lubatud velg W 9 ; W 10 Laius, mm 325 Läbimõõt, mm 1360 SLR, мм 639 RC, mm 3998 Kandevõime (kg) / km / h 1 1600 / 30 Rõhk bar 2.3 Kaal (kg) 48.74000 Mustri sügavus (mm) 34 TRA code R1 Maht, m3 0.45000\n
\n
URL: https://www.heuver.com/product/b12400032bk0813500/12-4-32-bkt-tr-135-124a6-121a8-8pr-tt\n
SNIPPET: EAN code. 8903094003235. Brand. BKT. Profile. TR-135. L/S. 124A6/121A8. Ply rating. 8. TL/TT. TT. Application. Diagonal. Radial/diagonal. Diagonal. Recommended ...Home Home Agricultural 12.4-32 BKT TR-135 124A6/121A8 8PR TT Home Home Agricultural 12.4-32 BKT TR-135 124A6/121A8 8PR TT This BKT 12.4-32 diagonal agricultural tyre will meet your needs! You have thus selected a pattern depth of 34 mm and a weight of 48.74 kg. It should preferably be used with a pressure of 2.3 bar. Zwedenweg 17 7772 TC Hardenberg The Netherlands Both we and third parties use cookies on our website.We use cookies to track statistics, save your preferences, for marketing purposes, and to personalise advertisements. Read more in our privacy statement 12.4-32 BKT TR-135 124A6/121A8 8PR TT Contact data We use cookies Back Necessary Preferences Analysis Advertising Products Truck Agriculture OTR Knowledge It's all round News About Heuver Contact Products Truck Agriculture OTR Knowledge It's all round News About Heuver Contact Home Home Agricultural Home Home Agricultural Login Register Why Heuver Our history Terms & Conditions Transport Conditions Knowledge News More About Heuver Login Register Forgot password Bonusshop It's all round Guarantees Claim form More My Heuver +31 (0)523 850 850 info@heuver.com More Contact data © 2024 Heuver.com Privacy statement Cookie statement Cookie settings Disclaimer Read more in our privacy statement Products Back Truck Agriculture OTR Truck Agriculture OTR Knowledge It's all round News About Heuver Contact Truck Agriculture OTR Knowledge It's all round News About Heuver Contact Why Heuver Our history Terms & Conditions Transport Conditions Knowledge News Login Register Forgot password Bonusshop It's all round Guarantees Claim form +31 (0)523 850 850 info@heuver.com Zwedenweg 17 7772 TC Hardenberg The Netherlands\n
\n
URL: https://shop.bohnenkamp-baltic.com/12-4-32-8pr-bkt-tr-135-124a6-r1-tt.html?___store=lt&___from_store=default\n
SNIPPET: 15722535 EAN: 8903094003235. Pneumatinė padanga 12.4-32 8PR BKT TR 135 124A6 R1 TT. Gerbiami klientai! Jeigu Jums reikalingų prekių atsargų stulpelyje matote ...pavyzdžiui 9524 padangos 9.5 24 dydžiui diagonalinė ar radialinė Krepšelis Pageidavimų sąrašas (0) Log In Šiuose puslapiuose visa informacija pagrįsta techninėmis gamintojo charakteristikomis. Turinys gali būti naudojamas tik informaciniais tikslais. Tiekėjas už šią informaciją neprisiima jokios atsakomybės . Atsakomybė už tiesioginius ar netiesioginius nuostolius, ieškiniai dėl žalos ar netiesioginės įvairaus pobūdžio žalos, susijusios su šios informacijos panaudojimu, negalimi. UAB «Bohnenkamp»Ukmergės g. 364 -15, Vilnius, Lietuva, LT-14188 Tel.: +370 52332611, +370 52332355 e-mail: info@bohnenkamp.lt www.bohnenkamp.lt UAB «Bohnenkamp»Ukmergės g. 364 -15, Vilnius, Lietuva, LT-14188 Tel.: +370 52332611, +370 52332355 e-mail: info@bohnenkamp.lt www.bohnenkamp.lt * Privalomas laukelis Jums reikalinga pagalba?Norėdami peržiūrėti vaizdinę medžiagą, paaiškinančią kaip naudotis internetine parduotuve, spustelėkite piktogramą, esančią dešinėje. 12.4-32 8PR BKT TR 135 124A6 R1 TT Related products Product number / Full text Size / Search code Information My account Contact Please select the customer offer to which you want to add this product Cart Wish list (0) 0 Log In EN EE LV RU Tires Cameras Discs Wheels Machine industry Other Special offers My shop Services First >> Add and open an offer Add and open an offer Add Add Cancel Cancel Photos EAN: 8903094003235 rim rim rim rim rim tire rim rim tube rim rim rim rim rim rim tire rim rim rim rim rim tire UAB «Bohnenkamp» Ukmergės st. 364 -15, Vilnius, Lithuania, LT-14188 Tel.: +370 52332611, +370 52332355 e-mail: info@bohnenkamp.lt www.bohnenkamp.lt @ Send Send Bohnenkamp - Profil für Profis Cart Wish list (0) Log In EN EE LV RU Tires Chambers Discs Wheels Machine industry Other Special offers My shop Services hier First rim 10.00X32 (-) 8/221/275 ET0 SILVER RAL9006 STARCO ONE PART 011900 - rim 10.00X32 (-) 10/281/335 ET0 -12 YELLOW RAL1018 STARCO 021477 - rim 10.00X32 6/163/222.3 ET-104 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 10/281/335 ET0 ORANGE RAL2002 STARCO 020711 - rim 10.00X32 (-) 8/221/275 Ø20 ET-60 YELLOW RAL1018 TITAN - tire 12.4-32 8PR BKT TR 171 124A6 R1W+ TT rim 10.00X32 (-) 8/221/275 ET50 ORANGE RAL2002 STARCO 010688 - rim 10X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS tube 11.2-32 Nexen TR218A 11.2/10-32, 12.4/11-32, 320/70-32, 360/70-32 rim 10.00X32 (-) 8/221/275 ET115 RED RAL3020 STARCO 020318 - rim 10.00X32 Ø17 ADJUSTABLE YELLOW RAL1018 TEJWHEELS COMBY rim 10.00X32 10/93/140 A2/21,5 ET30.5 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 8/221/275 ET90 YELLOW RAL1018 STARCO 010466 - rim 10.00X32 (-) 10/ 281/335 ET140 YELLOW RAL1018 STARCO 020956 - rim 10.00X32 (-) 10/281/335 Ø21 ET190 YELLOW RAL1018 STARCO 628486 - tire 320/85R32 (12.4R32) BKT AGRIMAX RT 855 126A8 /126B R1W TL 12 .4R32 rim 10.00X32 ( -) 8/221/275 Ø20 ET-20 RED RAL3020 STARCO 012064 - rim 10.00X32 ADJUSTABLE SILVER RAL9006 TEJWHEELS COMBY rim 10.00X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS TEJ01019 rim 10.00X32 (-) 1 0/ 281/335 ET50 ORANGE RAL2002 STARCO 020877 - rim 10.00X32 12/390/430 Ø17 ET-177 SILVER RAL9006 TEJWHEELS COMBY TEJ00527-525 tire 12.4-32 8PR BKT FARM SPECIAL 124A6 R1 TT Specification Downloads Properties Terms and Conditions Privacy Policy Cart mark 15722535 Tires Tubes Rims Wheels Machine industry Other Special offers My shop Services First >> 12.4-32 8PR BKT TR 135 124A6 R1 TT rim 10.00X32 (-) 8/221/275 ET0 SILVER RAL9006 STARCO ONE PART 011900 - rim 10.00X32 (-) 10/281/335 ET-12 YELLOW RAL1018 STARCO 021477 - rim 10.00X32 6/163/222.3 ET-104 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 10/281/335 ET0 ORANGE RAL2002 STARCO 020711 - rim 10.00X32 (-) 8/221/275 Ø20 ET-60 YELLOW RAL1018 TITAN - tire 12.4-32 8PR BKT TR 171 124A6 R1W+ TT rim 10.00X32 (-) 8/221/275 ET50 ORANGE RAL2002 STARCO 010688 - rim 10X32 8/152.4/203.2 A2/ 21,5 ET30 SILVER RAL9006 TEJWHEELS tube 11.2-32 Nexen TR218A 11.2/10-32, 12.4/11-32, 320/70-32, 360/70-32 rim 10.00X32 (-) 8/221/275 ET115 RED RAL3020 STARCO 020318 - rim 10.00X32 Ø17 ADJUSTABLE YELLOW RAL1018 TEJWHEELS COMBY rim 10.00X32 10/93/140 A2/21,5 ET30.5 SILVER RAL9006 TEJWHEELS rim 10.00X32 (-) 8/221/275 ET90 YELLOW RAL1018 STARCO 010466 - rim 10.00 X32 (-) 10/281/335 ET140 YELLOW RAL1018 STARCO 020956 - rim 10.00X32 (-) 10/281/335 Ø21 ET190 YELLOW RAL1018 STARCO 628486 - tire 320/85R32 (12.4R32) BKT AGRIMAX RT 855 126A8 /126B R1W TL 12.4R32 rim 10.00X32 (-) 8/221/275 Ø20 ET-20 RED RAL3020 STARCO 012064 - rim 10.00X32 ADJUSTABLE SILVER RAL9006 TEJWHEELS COMBY rim 10.00X32 8/152.4/203.2 A2/21,5 ET30 SILVER RAL9006 TEJWHEELS TEJWHEELS 1019 rim 10.00 X32 (-) 10/281/335 ET50 ORANGE RAL2002 STARCO 020877 - rim 10.00X32 12/390/430 Ø17 ET-177 SILVER RAL9006 TEJWHEELS COMBY TEJ00527-525 tire 12.4-32 8PR BKT FARM SPECIAL 124A6 R1 TT Specification Downloads SKU1572253 5 Size 12.4 -32TL / TTTTLI / SI124A6PR8Tread TR 135 ManufacturerBKTRecommended rimW 11Permissible rimW 9 ; W 10Width, mm325Diameter, mm1360SLR, mm639RC, mm3998Lifting power(kg)/ km/h 11600 / 30Pressure, bar2.3Weight kg48.74000Dug height mm34TRA codeR1Scope, m30.45000 On these pages, all information is based on the technical characteristics of the manufacturer. The content can be used for informational purposes only. The provider does not assume any responsibility for this information. Liability for direct or indirect losses, claims for damages or indirect damages of various kinds related to the use of this information is not possible. Details Terms and conditions Data protection Cart Mark *Name Phone *Email *Comment SKU 15722535 Size 12.4-32 TL / TT TT LI / SI 124A6 PR 8 Protector TR 135 Manufacturer BKT Recommended rim W 11 Permissible rim W 9 ; W 10 Width, mm 325 Diameter, mm 1360 SLR, mm 639 RC, mm 3998 Ke lifting power (kg)/ km/h 1 1600 / 30 Pressure, bar 2.3 Weight kg 48.74000 Height of ground supports mm 34 TRA code R1 Volume, m3 0.45000\n
"""

# define the llm model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.1,
    google_api_key=gemini_api_key
)

from langchain.chains import create_extraction_chain

# define the schema
custom_schema = {
    "properties": {
        "ean_code": {"type": "string"},
        "diameter": {"type": "float"},
        "width": {"type": "float"},
        "height": {"type": "float"},
        "price": {"type": "float"},
        "brand": {"type": "string"},
        "model": {"type": "string"},
        "vehicle_type": {"type": "string"}
    },
    "required": ["ean_code"]
}

# create the extraction chain
def  extract(content, schema):
    return create_extraction_chain(
        schema=schema,
        llm=llm
    ).invoke(content)

from pprint import pprint
# extract the information
extracted_content = extract(docs_text, custom_schema)
pprint(extracted_content)

ValueError: Unknown field for GenerateContentRequest: function_call

In [ ]:
# Resources
# https://python.langchain.com/docs/use_cases/web_scraping#scraping-with-extraction
# https://python.langchain.com/docs/use_cases/extraction#quickstart
# https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.refine.RefineDocumentsChain.html#langchain.chains.combine_documents.refine.RefineDocumentsChain
# https://python.langchain.com/docs/use_cases/summarization#option-3.-refine
# https://blog.stackademic.com/web-scraping-with-llms-using-langchain-8039b10a6de9?gi=64f7279043f8
# https://github.com/google/generative-ai-docs/tree/main/examples/gemini/python